In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
#pip install pandas openpyxl 

In [11]:
url = "https://raw.githubusercontent.com/jaemunoz8/TFM/refs/heads/main/DatosLimpiosAnonimizados.csv"
# Cargar el archivo como un DataFrame
datos_anonimizados = pd.read_csv(url)

# Mostrar las primeras filas del DataFrame
print(datos_anonimizados.head())

  NumeroFactura  FechaVenta    Precio  Cantidad ReferenciaProducto  \
0         fact1  2024-01-02  50411.76       1.0               ref1   
1         fact2  2024-01-02  17638.66       1.0               ref2   
2         fact3  2024-01-02      0.00       1.0               ref3   
3         fact4  2024-01-02   8394.96       1.0               ref4   
4         fact5  2024-01-02  33605.04       1.0               ref5   

  CodigoMarca IdUser          MedioPago   Talla   Color   Fabricante  \
0      marca1    id1  TARJETA DE DEBITO  talla1  color1  fabricante1   
1      marca2    id2           EFECTIVO  talla2  color2  fabricante1   
2      marca2    id3  TARJETA DE DEBITO  talla2  color3  fabricante1   
3      marca1    id4  TARJETA DE DEBITO  talla2  color4  fabricante2   
4      marca2    id5  TARJETA DE DEBITO  talla3  color5  fabricante1   

  ClaseProducto NombreTienda DepartamentoTienda  CiudadTienda  
0    claseprod1      tienda1    VALLE DEL CAUCA          CALI  
1    claseprod2   

In [12]:
#IRS BASADO EN USUARIOS
#Se genera primero la matriz de usuarios con los ítems con los que ha interactuado, en este caso que ha adquirido
df_filtrado3 = datos_anonimizados[[
    'IdUser', 'ReferenciaProducto', 'CodigoMarca', 
    'Precio', 'Cantidad'
]]

matrizItemUsuario = df_filtrado3.pivot_table(
    index='IdUser', 
    columns='ReferenciaProducto', 
    values='Cantidad', 
    aggfunc='sum',
    fill_value=0
)
print(matrizItemUsuario)
print(matrizItemUsuario.shape)

# Exportar el dataframe a un archivo Excel
# output_file_path = RutaSalida3 = "C:/Users/jaemu/Documents/_DatosTFM/MatrizItemUser.xlsx"
# matrizItemUsuario.T.to_excel(output_file_path, index=True)
#matrizItemUsuario.head()

ReferenciaProducto  ref1  ref10  ref100  ref1000  ref1001  ref1002  ref1003  \
IdUser                                                                        
id1                    1      0       0        0        0        0        0   
id10                   0      0       0        0        0        0        0   
id100                  0      0       0        0        0        0        0   
id1000                 0      0       0        0        0        0        0   
id10000                0      0       0        0        0        0        0   
...                  ...    ...     ...      ...      ...      ...      ...   
id9995                 0      0       0        0        0        0        0   
id9996                 0      0       0        0        0        0        0   
id9997                 0      0       0        0        0        0        0   
id9998                 0      0       0        0        0        0        0   
id9999                 0      0       0        0    

In [13]:
#Se calcula la matriz de similitud entre usuarios
user_similarity = cosine_similarity(matrizItemUsuario)

df_similitudUser = pd.DataFrame(user_similarity, index=matrizItemUsuario.index, columns=matrizItemUsuario.index)
np.round(df_similitudUser,2).iloc[:9,:9]

IdUser,id1,id10,id100,id1000,id10000,id10001,id10002,id10003,id10004
IdUser,,,,,,,,,
id1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
id100,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
id1000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
id10000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
id10001,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
id10002,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
id10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
id10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
def recomendar_productos_usuario(IdUser, matrizItemUsuario, df_similitudUser, top_n=5):

    if IdUser not in df_similitudUser.index:
        print(f"El usuario {IdUser} no se encuentra en la matriz de similitud.")
        return []
    
    # Obtener los usuarios más similares al usuario dado
    usuarios_similares = df_similitudUser.loc[IdUser].sort_values(ascending=False)
    usuarios_similares = usuarios_similares.drop(IdUser)  # Excluir al usuario actual
    
    # Obtener productos que han comprado los usuarios similares, ponderados por similitud
    productos_recomendados = pd.Series(dtype='float64')
    
    for usuario_similar, similitud in usuarios_similares.items():  # Cambiado iteritems() por items()
        # Productos comprados por el usuario similar
        productos_usuario = matrizItemUsuario.loc[usuario_similar]
        productos_usuario = productos_usuario[productos_usuario > 0]
        
        # Agregar los productos ponderados por la similitud
        productos_recomendados = productos_recomendados.add(productos_usuario * similitud, fill_value=0)
    
    # Remover los productos que el usuario ya compró
    productos_usuario_actual = matrizItemUsuario.loc[IdUser]
    productos_comprados = productos_usuario_actual[productos_usuario_actual > 0].index
    productos_recomendados = productos_recomendados.drop(productos_comprados, errors='ignore')
    
    # Ordenar las recomendaciones y devolver los `top_n`
    recomendaciones = productos_recomendados.sort_values(ascending=False).head(top_n)
    return recomendaciones.index.tolist()

# Se realiza una recomendación
IdUser_ejemplo = 'id11'
productos_recomendados = recomendar_productos_usuario(IdUser_ejemplo, matrizItemUsuario, df_similitudUser, top_n=5)
print(f"Productos recomendados para el usuario {IdUser_ejemplo}: {productos_recomendados}")


Productos recomendados para el usuario id11: ['ref327', 'ref83', 'ref195', 'ref217', 'ref648']


In [15]:
def recomendar_productos_usuario(IdUser, matrizItemUsuario, df_similitudUser, top_n=5):
    if IdUser not in df_similitudUser.index:
        print(f"El usuario {IdUser} no se encuentra en la matriz de similitud.")
        return []
    
    usuarios_similares = df_similitudUser.loc[IdUser].sort_values(ascending=False)
    usuarios_similares = usuarios_similares.drop(IdUser, errors='ignore')
    
    productos_recomendados = pd.Series(dtype='float64')
    
    for usuario_similar, similitud in usuarios_similares.items():
        if usuario_similar in matrizItemUsuario.index:
            productos_usuario = matrizItemUsuario.loc[usuario_similar]
            productos_usuario = productos_usuario[productos_usuario > 0]
            
            # Validar referencias existentes
            productos_usuario = productos_usuario[productos_usuario.index.isin(matrizItemUsuario.columns)]
            
            productos_recomendados = productos_recomendados.add(
                productos_usuario * similitud, fill_value=0
            )
    
    if IdUser in matrizItemUsuario.index:
        productos_usuario_actual = matrizItemUsuario.loc[IdUser]
        productos_comprados = productos_usuario_actual[productos_usuario_actual > 0].index
        
        # Eliminar solo productos válidos
        productos_recomendados = productos_recomendados.drop(
            [p for p in productos_comprados if p in productos_recomendados.index],
            errors='ignore'
        )
    
    recomendaciones = productos_recomendados.sort_values(ascending=False).head(top_n)
    return recomendaciones.index.tolist()

# Prueba con diagnóstico
IdUser_ejemplo = 'id11'
try:
    productos_recomendados = recomendar_productos_usuario(IdUser_ejemplo, matrizItemUsuario, df_similitudUser, top_n=5)
    print(f"Productos recomendados para el usuario {IdUser_ejemplo}: {productos_recomendados}")
except KeyError as e:
    print(f"Error de KeyError: {e}")


Productos recomendados para el usuario id11: ['ref327', 'ref83', 'ref195', 'ref217', 'ref648']


In [16]:
#Métricas de prueba
from sklearn.model_selection import train_test_split

# Separar en entrenamiento y prueba
train, test = train_test_split(df_filtrado3, test_size=0.2, random_state=42)

# Crear matriz de usuario-producto para entrenamiento
matrizItemUsuario_train = train.pivot_table(
    index='IdUser', 
    columns='ReferenciaProducto', 
    values='Cantidad', 
    aggfunc='sum',
    fill_value=0
)

# Crear matriz de prueba
matrizItemUsuario_test = test.pivot_table(
    index='IdUser', 
    columns='ReferenciaProducto', 
    values='Cantidad', 
    aggfunc='sum',
    fill_value=0
)


In [17]:
# Reducción de datos
def reducir_datos(df, n_usuarios=1000, umbral_interacciones_producto=10):
    # Seleccionar los usuarios más activos
    usuarios_activos = df.groupby('IdUser')['ReferenciaProducto'].count()
    usuarios_seleccionados = usuarios_activos.nlargest(n_usuarios).index

    # Filtrar usuarios seleccionados
    df_reducido = df[df['IdUser'].isin(usuarios_seleccionados)]

    # Filtrar productos con suficientes interacciones
    productos_populares = df_reducido['ReferenciaProducto'].value_counts()
    productos_filtrados = productos_populares[productos_populares >= umbral_interacciones_producto].index

    # Filtrar el DataFrame final
    df_reducido = df_reducido[df_reducido['ReferenciaProducto'].isin(productos_filtrados)]
    return df_reducido

# Separar en datos de entrenamiento y prueba
def preparar_datos(df):
    train, test = train_test_split(df, test_size=0.2, random_state=42)

    matrizItemUsuario_train = train.pivot_table(
        index='IdUser', 
        columns='ReferenciaProducto', 
        values='Cantidad', 
        aggfunc='sum',
        fill_value=0
    )

    matrizItemUsuario_test = test.pivot_table(
        index='IdUser', 
        columns='ReferenciaProducto', 
        values='Cantidad', 
        aggfunc='sum',
        fill_value=0
    )
    return matrizItemUsuario_train, matrizItemUsuario_test

# Función para recomendar productos a un usuario
def recomendar_productos_usuario(IdUser, matrizItemUsuario, df_similitudUser, top_n=5):
    if IdUser not in df_similitudUser.index:
        return []

    usuarios_similares = df_similitudUser.loc[IdUser].sort_values(ascending=False).drop(IdUser)

    productos_recomendados = pd.Series(dtype='float64')
    for usuario_similar, similitud in usuarios_similares.items():
        productos_usuario = matrizItemUsuario.loc[usuario_similar]
        productos_usuario = productos_usuario[productos_usuario > 0]
        productos_recomendados = productos_recomendados.add(productos_usuario * similitud, fill_value=0)

    productos_usuario_actual = matrizItemUsuario.loc[IdUser]
    productos_comprados = productos_usuario_actual[productos_usuario_actual > 0].index
    productos_recomendados = productos_recomendados.drop(productos_comprados, errors='ignore')

    recomendaciones = productos_recomendados.sort_values(ascending=False).head(top_n)
    return recomendaciones.index.tolist()

# Precision@K y Recall@K
def precision_recall_at_k(IdUser, recomendaciones, matrizItemUsuario_test, k=5):
    if IdUser not in matrizItemUsuario_test.index:
        return 0, 0

    relevantes = matrizItemUsuario_test.loc[IdUser]
    relevantes = relevantes[relevantes > 0].index

    predicciones = set(recomendaciones[:k])
    relevantes_recomendados = predicciones.intersection(relevantes)

    precision = len(relevantes_recomendados) / k
    recall = len(relevantes_recomendados) / len(relevantes) if len(relevantes) > 0 else 0
    return precision, recall

# Cobertura
def cobertura(recomendaciones_totales, total_productos):
    productos_recomendados = set()
    for recs in recomendaciones_totales.values():
        productos_recomendados.update(recs)
    return len(productos_recomendados) / total_productos

# Diversidad
def diversidad(recomendaciones_totales, matrizItemUsuario):
    productos = matrizItemUsuario.columns
    matriz_similitud = cosine_similarity(matrizItemUsuario.T)
    similitud_df = pd.DataFrame(matriz_similitud, index=productos, columns=productos)

    diversidad_total = []
    for recs in recomendaciones_totales.values():
        if len(recs) > 1:
            pares = [(a, b) for i, a in enumerate(recs) for b in recs[i+1:]]
            diversidad_total.extend([1 - similitud_df.loc[a, b] for a, b in pares if a in similitud_df.index and b in similitud_df.columns])

    return sum(diversidad_total) / len(diversidad_total) if diversidad_total else 0

# Evaluar el sistema de recomendación
def evaluar_sistema_recomendacion(matrizItemUsuario_train, matrizItemUsuario_test, df_similitudUser, top_n=5):
    usuarios = matrizItemUsuario_train.index

    precisiones, recalls = [], []
    recomendaciones_totales = {}

    for user in usuarios:
        recomendaciones = recomendar_productos_usuario(user, matrizItemUsuario_train, df_similitudUser, top_n=top_n)
        recomendaciones_totales[user] = recomendaciones

        precision, recall = precision_recall_at_k(user, recomendaciones, matrizItemUsuario_test, k=top_n)
        precisiones.append(precision)
        recalls.append(recall)

    precision_promedio = sum(precisiones) / len(precisiones)
    recall_promedio = sum(recalls) / len(recalls)

    cobertura_score = cobertura(recomendaciones_totales, len(matrizItemUsuario_train.columns))
    diversidad_score = diversidad(recomendaciones_totales, matrizItemUsuario_train)

    return {
        "Precision@K": precision_promedio,
        "Recall@K": recall_promedio,
        "Cobertura": cobertura_score,
        "Diversidad": diversidad_score,
    }

# Carga y reducción de datos
df_filtrado3 = datos_anonimizados[['IdUser', 'ReferenciaProducto', 'Cantidad']]
df_filtrado3 = reducir_datos(df_filtrado3)

# Preparación de datos
matrizItemUsuario_train, matrizItemUsuario_test = preparar_datos(df_filtrado3)

# Cálculo de similitudes
user_similarity = cosine_similarity(matrizItemUsuario_train)
df_similitudUser = pd.DataFrame(user_similarity, index=matrizItemUsuario_train.index, columns=matrizItemUsuario_train.index)

# Evaluación del sistema
resultados = evaluar_sistema_recomendacion(matrizItemUsuario_train, matrizItemUsuario_test, df_similitudUser, top_n=5)
print(resultados)


{'Precision@K': 0.0005649717514124294, 'Recall@K': 0.002824858757062147, 'Cobertura': 0.84, 'Diversidad': 0.9935271230807555}
